In [1]:
import os
import pytz
import pandas as pd
import geopandas as gpd
import contextily as ctx
from datetime import datetime
import matplotlib.pyplot as plt
from shapely.geometry import Point
from matplotlib import rcParams

# Headers for requests
headers = {
    'accept': 'application/json, text/plain, */*',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}

# Set timezone to UTC and then convert to America/Los_Angeles
utc_now = datetime.utcnow().replace(tzinfo=pytz.utc)
la_now = utc_now.astimezone(pytz.timezone('America/Los_Angeles'))

# Get current week number and date in Los Angeles time
week_number = la_now.isocalendar().week
today_la = la_now.strftime('%Y-%m-%d')

# Use timestamp for cacheBust
cache_bust = int(la_now.timestamp())

# Format the URL dynamically
url = f'https://data.lacity.org/api/views/e7h6-4a3e/rows.csv?fourfour=s49e-q6j2&cacheBust={cache_bust}&date={today_la}&accessType=DOWNLOAD'

In [2]:
locations = pd.read_csv(url, storage_options=headers)  # Ensure headers are being used if required

In [3]:
# Convert the string to UTC datetime specifying the format
locations['EventTime_UTC'] = pd.to_datetime(locations['EventTime_UTC'], format='%m/%d/%Y %I:%M:%S %p', utc=True)

# Convert from UTC to Los Angeles time
locations['EventTime_LA'] = locations['EventTime_UTC'].dt.tz_convert('America/Los_Angeles')

In [4]:
locations.columns = locations.columns.str.lower()
locations_df = locations.sort_values('eventtime_la', ascending=False).copy()

In [5]:
locations_df.head()

,spaceid,eventtime_utc,occupancystate,eventtime_la
3752,WV392,2024-12-01 16:43:17+00:00,VACANT,2024-12-01 08:43:17-08:00
1455,WV298,2024-12-01 16:43:13+00:00,VACANT,2024-12-01 08:43:13-08:00
534,WV100,2024-12-01 16:43:12+00:00,VACANT,2024-12-01 08:43:12-08:00
737,HO480,2024-12-01 16:43:11+00:00,VACANT,2024-12-01 08:43:11-08:00
376,WV106,2024-12-01 16:43:09+00:00,OCCUPIED,2024-12-01 08:43:09-08:00


---

In [6]:
inventory_gdf = gpd.read_file('data/parking_meters_inventory_latest.geojson')

In [7]:
inventory_gdf.head()

,spaceid,blockface,metertype,ratetype,raterange,meteredtimelimit,name,city,region,latitude,longitude,geometry
0,BH288,2000 1ST ST,Single-Space,FLAT,$1.00,1HR,Boyle Heights,los-angeles,eastside,34.045060,-118.214318,POINT (-118.21432 34.04506)
1,SC645B,12701 W VENTURA BLVD,Single-Space,FLAT,$1.00,2HR,Studio City,los-angeles,san-fernando-valley,34.144850,-118.410761,POINT (-118.41076 34.14485)
2,ED634,1001 S TOWNE AVE,Single-Space,FLAT,$0.50,2HR,Downtown,los-angeles,central-la,34.033269,-118.250355,POINT (-118.25035 34.03327)
3,MM163,600 S LA BREA AVE,Single-Space,FLAT,$1.00,2HR,Hancock Park,los-angeles,central-la,34.064086,-118.343897,POINT (-118.34390 34.06409)
4,SO152,4401 N VAN NUYS BLVD,Single-Space,FLAT,$1.00,2HR,Sherman Oaks,los-angeles,san-fernando-valley,34.151753,-118.448832,POINT (-118.44883 34.15175)


---

In [8]:
gdf = gpd.GeoDataFrame(pd.merge(inventory_gdf, locations_df, on='spaceid'))

In [9]:
# Define the mapping from neighborhood to area
area_mapping = {
    'Marina del Rey': 'Beach',
    'Venice': 'Beach',
    'Downtown': 'Downtown',
    'Chinatown': 'Downtown',
    'Hollywood': 'Hollywood',
    'Elysian Park': 'Dodgers',
    'San Pedro': 'Harbor',
    'Westwood': 'UCLA'
}

# Apply the mapping to create a new 'Area' column
gdf['region'] = gdf['name'].map(area_mapping)

In [10]:
places = list(gdf['name'].unique())
data_list = []

for place in places:
    place_gdf = gdf[gdf['name'] == place]
    total_spaces = len(place_gdf)
    counts = place_gdf['occupancystate'].value_counts()
    percentages = counts / total_spaces * 100  # Calculate percentages
    
    # Create a data row for this region
    data_row = {
        'region': place,
        'spaces': total_spaces,
        'unknown': counts.get('UNKNOWN', 0),
        'occupied': counts.get('OCCUPIED', 0),
        'vacant': counts.get('VACANT', 0),
        'ukn_pct': percentages.get('UNKNOWN', 0),
        'occ_pct': percentages.get('OCCUPIED', 0),
        'vac_pct': percentages.get('VACANT', 0)
    }
    data_list.append(data_row)

# Convert list of data rows to DataFrame
summary_df = pd.DataFrame(data_list).round(2)

In [11]:
summary_df['fetched'] = pd.to_datetime(la_now).strftime('%Y-%m-%d %H:%M:%S')

In [12]:
summary_df

,region,spaces,unknown,occupied,vacant,ukn_pct,occ_pct,vac_pct,fetched
0,Downtown,3636,0,2090,1546,0.00,57.48,42.52,2024-12-01 08:44:38
1,San Pedro,293,0,192,101,0.00,65.53,34.47,2024-12-01 08:44:38
2,Hollywood,901,175,264,462,19.42,29.30,51.28,2024-12-01 08:44:38
3,Chinatown,472,0,310,162,0.00,65.68,34.32,2024-12-01 08:44:38
4,Venice,195,77,47,71,39.49,24.10,36.41,2024-12-01 08:44:38
5,Westwood,421,0,187,234,0.00,44.42,55.58,2024-12-01 08:44:38
6,Marina del Rey,28,22,3,3,78.57,10.71,10.71,2024-12-01 08:44:38
7,Elysian Park,13,0,11,2,0.00,84.62,15.38,2024-12-01 08:44:38


In [13]:
# Setting the font globally
rcParams['font.family'] = 'Roboto'

# Ensure the visuals directory exists
visuals_dir = 'visuals'
os.makedirs(visuals_dir, exist_ok=True)

for place in places:
    place_gdf = gdf.query(f'name == "{place}"').copy()
    place_gdf = place_gdf.to_crs(epsg=3857)  # Convert to Web Mercator

    # Create a dictionary to map occupancystate to colors
    color_map = {
        'UNKNOWN': '#666',
        'OCCUPIED': '#FF474C',
        'VACANT': '#90EE90'
    }
    
    # Map the colors to your dataframe
    place_gdf['color'] = place_gdf['occupancystate'].map(color_map)
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    # Plot the data using longitude and latitude as X and Y
    scatter = ax.scatter(x=place_gdf.geometry.x, y=place_gdf.geometry.y, alpha=1, s=20, c=place_gdf['color'], label='Occupancy State')
    
    # Add the basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    
    # Adjust the axis to fit the plot (this fixes any misalignment issues)
    ax.set_xlim(place_gdf.geometry.bounds.minx.min(), place_gdf.geometry.bounds.maxx.max())
    ax.set_ylim(place_gdf.geometry.bounds.miny.min(), place_gdf.geometry.bounds.maxy.max())

    # Remove the axis
    ax.set_axis_off()

    # Setting title with specific font
    ax.set_title(f'{place} Parking Meter Locations', fontsize=14)
    
    # Create a custom legend
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label='Unknown', markersize=10, markerfacecolor='#666'),
                       plt.Line2D([0], [0], marker='o', color='w', label='Occupied', markersize=10, markerfacecolor='#FF474C'),
                       plt.Line2D([0], [0], marker='o', color='w', label='Vacant', markersize=10, markerfacecolor='#90EE90')]
    ax.legend(handles=legend_elements, title='Occupancy State')

    # Save the figure
    fig_path = os.path.join(visuals_dir, f'{place.replace(" ", "_").lower()}_parking_meter_locations.png')
    plt.savefig(fig_path, format='png', dpi=300)
    plt.close(fig)  # Close the plot to free up memory